In [1]:
import yaml
import os
from dataclasses import dataclass
from pathlib import Path
import torchio as tio

/Users/anirudhsharma/Desktop/Project/Identification_of_kidney_stones_empowered_with_XAI/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/anirudhsharma/Desktop/Project/Identification_of_kidney_stones_empowered_with_XAI'

In [4]:
@dataclass(frozen=True)
class PreprocessingConfig:
    root_dir: Path
    processed_data_path: Path
    params_clahe_clip: float
    params_denoise_strength: float
    params_intensity_rescale: bool

In [10]:
from KidneyStoneClassification.constants import *
from KidneyStoneClassification import logger
from KidneyStoneClassification.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_preprocessing_config(self) -> PreprocessingConfig:
        config = self.config.preprocessing
        
        create_directories([config.root_dir])

        preprocessing_config = PreprocessingConfig(
            root_dir=Path(config.root_dir),
            processed_data_path=Path(config.processed_data_path),
            params_clahe_clip=self.params.CLAHE_CLIP,
            params_denoise_strength=self.params.DENOISE_STRENGTH,
            params_intensity_rescale=self.params.INTENSITY_RESCALE
        )

        return preprocessing_config

In [13]:
import cv2
import numpy as np
from tqdm import tqdm

In [14]:
class Preprocessing:
    def __init__(self, config: PreprocessingConfig):
        self.config = config

    def apply_clahe(self, image):
        clahe = cv2.createCLAHE(clipLimit=self.config.params_clahe_clip, tileGridSize=(8,8))
        return clahe.apply(image)

    def denoise(self, image):
        return cv2.fastNlMeansDenoising(image, None, self.config.params_denoise_strength, 7, 21)

    def intensity_rescale(self, image):
        return cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    
    def process_image(self, image_path):
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Error: Input image not found at {image_path}")

        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"Error: Failed to load image. Ensure {image_path} is a valid grayscale image.")

        image = self.apply_clahe(image)
        image = self.denoise(image)
        if self.config.params_intensity_rescale:
            image = self.intensity_rescale(image)
        return image
    
    def save_processed_image(self, image, output_path: str):
        output_dir = os.path.dirname(output_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir, exist_ok=True)
            
        cv2.imwrite(output_path, image)
        if not os.path.exists(output_path):
            logger.error(f"Error: Image was not saved to {output_path}")
            
    def process_directory(self, input_dir, output_dir):
        """
        Process all images in a directory structure, maintaining the same organization
        
        Args:
            input_dir: Root directory containing class folders
            output_dir: Output directory where processed images will be saved
        """
        logger.info(f"Starting to process images from {input_dir}")
        os.makedirs(output_dir, exist_ok=True)
        
        # Find all class folders
        class_folders = [d for d in os.listdir(input_dir) 
                         if os.path.isdir(os.path.join(input_dir, d))]
        
        total_processed = 0
        total_errors = 0
        
        for class_name in class_folders:
            logger.info(f"Processing class: {class_name}")
            class_path = os.path.join(input_dir, class_name)
            output_class_path = os.path.join(output_dir, class_name)
            os.makedirs(output_class_path, exist_ok=True)
            
            image_files = [f for f in os.listdir(class_path) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            for img_file in tqdm(image_files, desc=f"Processing {class_name}"):
                try:
                    input_path = os.path.join(class_path, img_file)
                    output_path = os.path.join(output_class_path, img_file)
                    
                    # Process and save the image
                    processed_img = self.process_image(input_path)
                    self.save_processed_image(processed_img, output_path)
                    total_processed += 1
                    
                except Exception as e:
                    logger.error(f"Error processing {img_file}: {str(e)}")
                    total_errors += 1
        
        logger.info(f"Processing complete! Processed {total_processed} images with {total_errors} errors")
        return total_processed, total_errors

In [16]:
try:
    from KidneyStoneClassification.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
    from KidneyStoneClassification.utils.common import read_yaml, create_directories
    
    class ConfigurationManager:
        def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
            
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
            
        def get_preprocessing_config(self) -> PreprocessingConfig:
            config = self.config.preprocessing
            create_directories([config.root_dir])
            
            preprocessing_config = PreprocessingConfig(
                root_dir=Path(config.root_dir),
                processed_data_path=Path(config.processed_data_path),
                params_clahe_clip=self.params.CLAHE_CLIP,
                params_denoise_strength=self.params.DENOISE_STRENGTH,
                params_intensity_rescale=self.params.INTENSITY_RESCALE
            )
            
            return preprocessing_config
            
    # Get configuration
    config = ConfigurationManager()
    preprocessing_config = config.get_preprocessing_config()
    
    # Initialize preprocessing
    preprocessing = Preprocessing(config=preprocessing_config)
    
    # Define input and output directories
    input_dir = "artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
    output_dir = "artifacts/data_preprocessing/processed_dataset"
    
    # Process the entire dataset
    preprocessing.process_directory(input_dir, output_dir)
    
    # Optionally, process a single image as an example
    sample_image = os.path.join(input_dir, "Stone", "Stone- (1).jpg")
    sample_output = os.path.join(output_dir, "sample_processed.jpg")
    
    if os.path.exists(sample_image):
        logger.info(f"Processing sample image: {sample_image}")
        processed_sample = preprocessing.process_image(sample_image)
        preprocessing.save_processed_image(processed_sample, sample_output)
        logger.info(f"Image processed and saved to {sample_output}")
    
except Exception as e:
    logger.error(f"Exception occurred: {str(e)}")

[2025-03-09 22:06:35,327: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-09 22:06:35,333: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-09 22:06:35,336: INFO: common: created directory at: artifacts]
[2025-03-09 22:06:35,336: INFO: common: created directory at: artifacts/preprocessing]
[2025-03-09 22:06:35,337: INFO: 2866771524: Starting to process images from artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone]
[2025-03-09 22:06:35,338: INFO: 2866771524: Processing class: Tumor]


Processing Tumor: 100%|██████████| 2283/2283 [02:49<00:00, 13.44it/s]

[2025-03-09 22:09:25,309: INFO: 2866771524: Processing class: Cyst]



Processing Cyst: 100%|██████████| 3709/3709 [04:48<00:00, 12.87it/s]

[2025-03-09 22:14:13,451: INFO: 2866771524: Processing class: Stone]



Processing Stone: 100%|██████████| 1377/1377 [01:58<00:00, 11.57it/s]

[2025-03-09 22:16:12,422: INFO: 2866771524: Processing class: Normal]



Processing Normal: 100%|██████████| 5077/5077 [09:37<00:00,  8.79it/s]   

[2025-03-09 22:25:50,123: INFO: 2866771524: Processing complete! Processed 12446 images with 0 errors]
[2025-03-09 22:25:50,124: INFO: 4028401034: Processing sample image: artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone/Stone- (1).jpg]


[2025-03-09 22:25:50,199: INFO: 4028401034: Image processed and saved to artifacts/data_preprocessing/processed_dataset/sample_processed.jpg]
